# RoBERTa in Healthcare – Hands‑on Colab Notebook
This notebook gives healthcare professionals a quick tour of what **RoBERTa** and its task‑specific fine‑tuned checkpoints can do:
1. Text (sentiment) classification
2. Clinical named‑entity recognition (NER)
3. Question answering over clinical notes
4. Semantic similarity / embeddings

Run each cell sequentially on **Google Colab** (GPU optional but recommended).

## 🔧 Setup – install libraries
Colab usually has PyTorch pre‑installed. We add 🤗 Transformers and 🧬 Sentence‑Transformers for embedding similarity.

In [ ]:
!pip install -q transformers torch sentencepiece sentence-transformers

## 1️⃣ Text Classification
Classify whether example discharge notes sound positive (recovery, stable) or concerning (complications, risk).

In [ ]:
from transformers import pipeline
import torch, textwrap

# General RoBERTa sentiment model
classifier = pipeline('text-classification', model='roberta-base',
                      device=0 if torch.cuda.is_available() else -1)

notes = [
    "The patient's vitals are stable and she is ready for discharge today.",
    "Despite medication, the patient's blood pressure remains dangerously high."
]

for note in notes:
    print('\n', '\n'.join(textwrap.wrap(note, width=80)))
    print('➡️', classifier(note)[0])

## 2️⃣ Named‑Entity Recognition (drugs, diseases, anatomy)
We switch to a checkpoint already fine‑tuned for English NER.

In [ ]:
ner = pipeline('ner', model='dslim/roberta-base-NER', grouped_entities=True,
               device=0 if torch.cuda.is_available() else -1)
sample = "The patient was prescribed 100 mg Metformin twice daily to manage type 2 diabetes."

ner(sample)

## 3️⃣ Question Answering on Clinical Notes
Ask specific questions against unstructured records. Useful for fast chart reviews.

In [ ]:
qa = pipeline('question-answering', model='deepset/roberta-base-squad2',
              device=0 if torch.cuda.is_available() else -1)

context = """\
Mr. Smith is a 67-year-old male with a history of COPD who was admitted due to shortness of breath. 
He was treated with bronchodilators and corticosteroids and showed significant improvement. 
He will be discharged with instructions to continue inhaled tiotropium once daily.
"""
question = "What medication should Mr. Smith continue after discharge?"

qa(question=question, context=context)

## 4️⃣ Semantic Similarity of Clinical Sentences
Turn sentences into dense vectors and compute cosine similarity – handy for deduplicating notes or matching similar patient cases.

In [ ]:
from sentence_transformers import SentenceTransformer, util
embedder = SentenceTransformer('sentence-transformers/all-roberta-large-v1')

s1 = "The patient complains of chest pain radiating to the left arm."
s2 = "Patient reports angina symptoms with discomfort spreading to the left arm."

e1, e2 = embedder.encode([s1, s2], convert_to_tensor=True)
print('Cosine similarity:', util.cos_sim(e1, e2).item())